## Using Schemas

In [1]:
SELECT * FROM public.customers LIMIT 5;

(5 row(s) affected)

Total execution time: 00:00:00.367

customerid,firstname,lastname,address1,address2,city,state,zip,country,region,email,phone,creditcardtype,creditcard,creditcardexpiration,username,password,age,income,gender
1,RNPOJG,FQIVBKWIZC,1287944454 Dell Way,,XCXNWIS,ND,42572,US,1,FQIVBKWIZC@dell.com,1287944454,5,1890860486775636,2012/06,user1,password,49,80000,M
2,RKLQYY,HPUIUYBWZX,3104451734 Dell Way,,OWLUWVE,CA,67434,US,1,HPUIUYBWZX@dell.com,3104451734,3,1066779058426618,2011/06,user2,password,87,40000,M
3,GWPSHI,QWRLBUPCHY,3471147404 Dell Way,,RYWSJPW,CT,49571,US,1,QWRLBUPCHY@dell.com,3471147404,1,5555479255504906,2010/09,user3,password,63,20000,M
4,ONCPHI,TZIJOMZQJJ,5811325555 Dell Way,,IATPPWH,AK,99909,US,1,TZIJOMZQJJ@dell.com,5811325555,5,9358483086340128,2008/06,user4,password,71,40000,M
5,CIDXWX,GNYGKZXSCR,9227139292 Dell Way,,WGHZCLX,WA,19937,US,1,GNYGKZXSCR@dell.com,9227139292,2,8126189742702991,2008/04,user5,password,18,100000,M


---
### Create a schema for customer objects and set up basic permissions

In [8]:
CREATE ROLE cust_owner;
CREATE SCHEMA cust AUTHORIZATION cust_owner;
GRANT USAGE ON SCHEMA cust to cust_owner;

ALTER DEFAULT PRIVILEGES 
    IN SCHEMA cust
    GRANT SELECT, INSERT, UPDATE, DELETE ON TABLES TO cust_owner;

---
### Copy customers table to new schema

In [9]:
CREATE TABLE cust.customers AS
    SELECT * FROM public.customers;

---
### Create function to read name from the customers table

In [2]:
CREATE OR REPLACE FUNCTION cust.get_cust_name(id INTEGER) 
    RETURNS VARCHAR
    LANGUAGE SQL
    STABLE LEAKPROOF
    SET search_path = cust
    SECURITY DEFINER
AS $$
    SELECT CONCAT_WS(' ', firstname, lastname) 
    FROM cust.customers c
    WHERE c.customerid = id;
$$;

ALTER FUNCTION cust.get_cust_name(id INTEGER)
    OWNER to cust_owner;

---
### Create report analyst role and users and revoke privileges

In [3]:
CREATE GROUP report_analysts;
CREATE USER Lindsay IN GROUP report_analysts PASSWORD 'p455w0rd';
GRANT USAGE ON SCHEMA cust TO report_analysts;

REVOKE SELECT, INSERT, UPDATE, DELETE 
    ON ALL TABLES IN SCHEMA cust FROM report_analysts;
GRANT EXECUTE ON ALL FUNCTIONS IN SCHEMA cust TO report_analysts;

---
### Clean Up

In [1]:
DROP FUNCTION IF EXISTS cust.get_cust_name(INT);
DROP TABLE IF EXISTS cust.customers;
DROP SCHEMA IF EXISTS cust;
DROP ROLE IF EXISTS cust_owner;
DROP USER IF EXISTS Lindsay;
DROP GROUP IF EXISTS report_analysts;